In [3]:
import tensorflow as tf
import numpy as np
import cv2
import os

In [4]:
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (128,128))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('dataset')
print('loaded',len(samples),' samples')

print('all classes',set(labels))

labels[labels=='new_cats']='cats'
labels[labels=='new_dogs']='dogs'

print('classes',set(labels))

# normalization
samples = samples / 255.0

loaded 6192  samples
all classes {'dogs', 'new_dogs', 'cats', 'new_cats'}
classes {'dogs', 'cats'}


In [5]:
from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder() # encoder
intlabels = encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(intlabels)
print("One-hot labels",labels.shape)

One-hot labels (6192, 2)


In [6]:
from tensorflow.python.keras.models import load_model
model = load_model("final_model.h5") ### add the correct model name!!!
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      448       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0

In [7]:
testSamples = samples
testLabels = labels

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
testResults = model.predict(testSamples)

print(confusion_matrix(testLabels.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(testLabels.argmax(axis=1), testResults.argmax(axis=1)))
print("CNN Accuracy: {:.2f}".format(accuracy_score(testLabels.argmax(axis=1), testResults.argmax(axis=1))))
print("Cohen's Kappa: {:.2f}".format(cohen_kappa_score(testLabels.argmax(axis=1), testResults.argmax(axis=1))))


[[2955  140]
 [ 243 2854]]
              precision    recall  f1-score   support

           0       0.92      0.95      0.94      3095
           1       0.95      0.92      0.94      3097

    accuracy                           0.94      6192
   macro avg       0.94      0.94      0.94      6192
weighted avg       0.94      0.94      0.94      6192

CNN Accuracy: 0.94
Cohen's Kappa: 0.88
